<a href="https://colab.research.google.com/github/cyberfork2000/pgz-campug/blob/master/MachineLearning_Course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import argparse, torch, torch.nn as nn
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# 1: Preparing The Data
PyTorch packages some common datasets like CIFAR10 nicely.  
A `DataLoader` is a PyTorch object that prepares data so it can be fed easily into a model.

In [0]:
training_data = DataLoader(
                    datasets.CIFAR10('data', train=True, download=True, transform=transforms.ToTensor()),
                    batch_size=128, shuffle=True, drop_last=True)

test_data = DataLoader(
                    datasets.CIFAR10('data', train=False, transform=transforms.ToTensor()),
                    batch_size=1000, shuffle=True)

In [0]:
train_batch = next(iter(training_data))
imgs, labels = train_batch

# Plot one of the images
plt.imshow(imgs[0].permute(1, 2, 0))
# Print the corresponding label
print(labels[0])

# 2: Neural Networks Making Predictions

In [0]:
class OurFirstNeuralNetwork(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(32*32*3, 10)
        
    def forward(self, img):
        x = img.view(-1, 32*32*3) # Flatten 2d image into 1d
        x = self.linear(x)
        return x

In [0]:
model = OurFirstNeuralNetwork()
predictions = model.forward(imgs)
predictions

In [0]:
predictions[0]

# 3: How Good Are Our Predictions?

In [0]:
def get_accuracy_on_batch(model, batch):
    total_correct_predictions = 0
    imgs, true_labels = batch
    
    predictions = model.forward(imgs)
    
    pred_labels = predictions.argmax(dim=1)
    total_correct_predictions += pred_labels.eq(true_labels.view_as(pred_labels)).sum().item()
    
    num_predictions = len(pred_labels)
    percentage_accuracy = (total_correct_predictions / num_predictions) * 100
    return percentage_accuracy

In [0]:
test_batch = next(iter(test_data))  # Get some data for testing
get_accuracy_on_batch(model, test_batch)

# 4: The Training Loop

In [0]:
def training_step(batch, model, optimizer):
    
    imgs, true_labels = batch
    
    imgs = imgs.cuda()
    true_labels = true_labels.cuda()
    
    predictions = model.forward(imgs)
    
    loss = nn.CrossEntropyLoss()(predictions, true_labels)
    
    optimizer.zero_grad()
    loss.backward()
    
    optimizer.step()
    
    # This function does not need to return anthing but we will return the loss for logging.
    return loss # Add this bit on the fly after having done the first bit of training.

In [0]:
model = OurFirstNeuralNetwork().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [0]:
for batch in training_data:
    training_step(batch, model, optimizer)

Same thing but add some logging

In [0]:
for i, batch in enumerate(training_data):
    loss = training_step(batch, model, optimizer)    
    
    if i % 100 == 0: print('Training Loss: {:.3f}'.format(loss.item()))

# 5: Putting It All Together

We now have everything we need to train a model and test its accuracy.

In [0]:
model = OurFirstNeuralNetwork().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


# Train the model
for epoch in range(3):
    
    for i, batch in enumerate(training_data):
        loss = training_step(batch, model, optimizer)    

        if i % 100 == 0: print('Training Loss: {:.3f}'.format(loss.item()))


# Check the model's accuracy on the test data
test_batch = next(iter(test_data))
get_accuracy_on_batch(model.cpu(), test_batch)

# 6: Deep Learning

In [0]:
class DeepNeuralNetwork(nn.Module):
  
  def __init__(self):
    super().__init__()
    self.linear_1 = nn.Linear(32*32*3, 1234)
    self.linear_2 = nn.Linear(1234, 1001)
    self.linear_3 = nn.Linear(1001, 10)
    
  def forward(self, x):
    x = x.view(-1, 32*32*3)
    x = self.linear_1(x)
    x = nn.ReLU()(x)
    x = self.linear_2(x)
    x = nn.ReLU()(x)
    x = self.linear_3(x)
    return x

In [0]:
model = DeepNeuralNetwork().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(3):
    
    for batch_num, batch in enumerate(training_data):
        loss = training_step(batch, model, optimizer)    

        if batch_num % 100 == 0: print('Training Loss: {:.3f}'.format(loss.item()))
        
test_batch = next(iter(test_data))
get_accuracy_on_batch(model.cpu(), test_batch)

# 7: CHALLENGER APPROACHING: Convolutional Neural Networks

In [0]:
class SimpleCNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3, 32, kernel_size=3, stride=2)
        self.linear = nn.Linear(7200, 10)
      
    def forward(self, x):
        x = self.conv(x)
        x = nn.ReLU()(x)
        x = x.view(-1, 7200)
        x = self.linear(x)
        return x

In [0]:
model = SimpleCNN().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(3):
    
    for batch_num, batch in enumerate(training_data):
        loss = training_step(batch, model, optimizer)    

        if batch_num % 100 == 0: print('Training Loss: {:.3f}'.format(loss.item()))
        
test_batch = next(iter(test_data))
get_accuracy_on_batch(model.cpu(), test_batch)

In [0]:
class DeepCNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv_1 = nn.Conv2d(3, 32, kernel_size=3, stride=2)
        self.conv_2 = nn.Conv2d(32, 32, kernel_size=3, stride=2)
        self.conv_3 = nn.Conv2d(32, 32, kernel_size=3, stride=2)
        self.conv_4 = nn.Conv2d(32, 32, kernel_size=3, stride=2)
        self.linear = nn.Linear(32, 10)
      
    def forward(self, x):
        
        x = self.conv_1(x)
        x = nn.ReLU()(x)
        x = self.conv_2(x)
        x = nn.ReLU()(x)
        x = self.conv_3(x)
        x = nn.ReLU()(x)
        x = self.conv_4(x)
        x = nn.ReLU()(x)
        
        x = x.view(-1, 32) 
        x = self.linear(x)
        return x

In [0]:
model = DeepCNN().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    
    for batch_num, batch in enumerate(training_data):
        loss = training_step(batch, model, optimizer)    

        if batch_num % 100 == 0: print('Training Loss: {:.3f}'.format(loss.item()))
        
test_batch = next(iter(test_data))
get_accuracy_on_batch(model.cpu(), test_batch)